### Ingest circuits.csv file

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
raw_folder_path

'/mnt/formula1dlajay/raw'

In [0]:
#widgits

dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
v_data_source

'testing'

##### Step 1 - Read the CSV file using the spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
circuits_schema = StructType(fields=[StructField('circuitId',IntegerType(),False),
                                     StructField('circuitRef',StringType(),True),
                                     StructField('name',StringType(),True),
                                     StructField('location',StringType(),True),
                                     StructField('country',StringType(),True),
                                     StructField('lat',DoubleType(),True),
                                     StructField('lng',DoubleType(),True),
                                     StructField('alt',IntegerType(),True),
                                     StructField('url',StringType(),True)])


In [0]:
circuits_df = spark.read.option("header",True).schema(circuits_schema).csv(f'{raw_folder_path}/circuits.csv')

In [0]:
from pyspark.sql.functions import col

In [0]:
circuits_selected_df = circuits_df.select(col("circuitId"),col("circuitRef"),col("name"),col("location"),col("country"),col("lat"),col("lng"),col("alt"))


#### Step -3 Remaned the Column as required


In [0]:
from pyspark.sql.functions import lit

In [0]:
circuits_renamed_df =  circuits_selected_df.withColumnRenamed("circuitId","circuit_id") \
                       .withColumnRenamed("circuitRef","circuit_ref") \
                       .withColumnRenamed("lat","latitude") \
                       .withColumnRenamed("lng","longitude") \
                       .withColumnRenamed("alt","altitude") \
                       .withColumn("data_source",lit(v_data_source))
                   


#### step -4 Add ingestion data to Dataframe

In [0]:
circuits_final_df = add_ingestion_date(circuits_renamed_df)


#### write data to datalake as parquet format

In [0]:
circuits_final_df.write.mode("overwrite").parquet(f'{processed_folder_path}/circuits') #to avoide error from rerun we use mode

In [0]:
# display(df)
display(spark.read.parquet(f'{processed_folder_path}/circuits/'))

circuit_id,circuit_ref,name,location,country,latitude,longitude,altitude,data_source,ingestion_date
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,testing,2024-01-27T04:27:51.359Z
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,testing,2024-01-27T04:27:51.359Z
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,testing,2024-01-27T04:27:51.359Z
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,testing,2024-01-27T04:27:51.359Z
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,testing,2024-01-27T04:27:51.359Z
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,testing,2024-01-27T04:27:51.359Z
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,testing,2024-01-27T04:27:51.359Z
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,testing,2024-01-27T04:27:51.359Z
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,testing,2024-01-27T04:27:51.359Z
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,103,testing,2024-01-27T04:27:51.359Z


In [0]:
dbutils.notebook.exit("Success")